# Transformer From Scratch

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pdb, math

import numpy as np
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context(context="talk")
%matplotlib inline

import torch;
assert(torch.cuda.is_available())
from torch import nn
from torch.nn import functional as F

In [3]:
from transformer import *

In [5]:
emb_size = 6
batch_size = 2
seq_len = 3
key_size,query_size,hidden_units,dropout = emb_size,emb_size,10,0.1
attn = MLPAttention(key_size, query_size, hidden_units, dropout).eval()
x = torch.rand(batch_size, seq_len, emb_size)

queries,keys,values = x,x,x
valid_len = torch.tensor([2, 6])

In [6]:
attn.forward(queries, keys, values, valid_len)

tensor([[[0.4910, 0.4504, 0.7237, 0.3687, 0.4688, 0.7265],
         [0.4911, 0.4503, 0.7236, 0.3685, 0.4689, 0.7262],
         [0.4909, 0.4504, 0.7237, 0.3688, 0.4688, 0.7266]],

        [[0.3827, 0.3301, 0.4166, 0.7643, 0.2257, 0.6270],
         [0.3817, 0.3319, 0.4159, 0.7642, 0.2255, 0.6274],
         [0.3797, 0.3312, 0.4145, 0.7643, 0.2260, 0.6260]]],
       grad_fn=<BmmBackward0>)